In [1]:
from hotels_50k import Hotels50KDataset

In [6]:
%load_ext autoreload
%autoreload 2

In [9]:
from torchvision import datasets

In [ ]:
folder = '/data/thesis/Hotels-50K/symlinks/train/chains/-1'
ds = datasets.ImageFolder(folder)

In [8]:
dataset = Hotels50KDataset(root='~/Documents/thesis/Hotels-50K', target='chains', download=False)

Loading image folders


KeyboardInterrupt: 

In [ ]:
from powerful_benchmarker.split_managers import ClassDisjointSplitManager, BaseSplitManager, IndexSplitManager
from torchvision import transforms
import torch
import numpy as np

class CustomSplitManager(BaseSplitManager):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.split_names = ["test"]

    def _create_split_schemes(self, datasets):
        output = {}
        for transform_type, v1 in datasets.items():
            output[transform_type] = {}
            for split_name, v2 in v1.items():
                indices = v2.get_split_indices(split_name)
                if indices is not None:
                    output[transform_type][split_name] = torch.utils.data.Subset(v2, indices)
                else:
                    output[transform_type][split_name] = v2
        return {self.get_split_scheme_name(0): output}

    def get_test_set_name(self):
        return 'UsingOriginalTest'

    def get_base_split_scheme_name(self):
        return self.get_test_set_name()

    def get_split_scheme_name(self, partition):
        return self.get_base_split_scheme_name()

    def split_assertions(self):
        pass


dataset_root = '/data/thesis/Hotels-50K'
the_dataset = Hotels50KDataset

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": the_dataset(dataset_root),
                    "val": the_dataset(dataset_root),
                    "test": the_dataset(dataset_root)}
            }

helper_split_manager = CustomSplitManager()
split_manager = IndexSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))